#### Importing libraries

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
import string
import matplotlib.pyplot as plt
import nltk

# importing dataset as fake and real

In [4]:
data_fake = pd.read_csv('fake.csv')
data_real = pd.read_csv('real.csv')

In [5]:
data_real.head()

,Title,Description
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,फिल्म ‘भगवत् गीता’मा नायक धिरज मगर अनुबन्धित ह...
1,पब्जी विश्वकपमा नेपाल दोस्रो,पब्जी विश्वकपमा नेपाल दोस्रो भएको छ । इन्डोनेस...
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,भारतका प्रधानमन्त्री नरेन्द्र मोदीले आफ्नी आमा...
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,कांग्रेसको १४ औं महाधिवेशमा नेतृत्व चयनका लागि...
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,यस वर्ष पहिचान पुरस्कार श्याम शाहको कथासंग्रह ...


In [6]:
data_fake.head()

,Title,Description
0,ट्रम्पडम: ट्रम्पको विदेश नीतिको जिज्ञासु संसा...,निरज श्रीवास्तव 21st Century WireIt डोनाल्ड ट्...
1,BREAKING: वफादार ट्रम्प समर्थकले महान्यायाधिवक...,"सेन जेफरसन ब्यूरेगार्ड सेसन्स III, डोनाल्ड ट्र..."
2,चीन ड्रोन ड्रामा ट्रम्पको पहिलो ट्वीट भन्दा पह...,डोनाल्ड ट्रम्पले अमेरिकी अन्डरवाटर ड्रोन जफत ग...
3,एक रक्षात्मक जोन केरीले सोध्छ कि इजरायलले आफ्न...,जोन केरीले ओबामा प्रशासन इजरायलको मित्र भएको द...
4,Reince Priebus ट्रम्पको दाबीलाई समर्थन गर्दछ क...,डोनाल्ड ट्रम्पले स्वतन्त्र प्रेस बिरूद्ध युद्ध...


# clasify gareu data fake lai 0 le real lai 1 lw

In [8]:
data_fake["class"]= 0
data_real["class"]= 1

# 2 tai data lai concatinate gareu

In [10]:
data = pd.concat([data_real,data_fake],axis=0)
data.head()

,Title,Description,class
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,फिल्म ‘भगवत् गीता’मा नायक धिरज मगर अनुबन्धित ह...,1
1,पब्जी विश्वकपमा नेपाल दोस्रो,पब्जी विश्वकपमा नेपाल दोस्रो भएको छ । इन्डोनेस...,1
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,भारतका प्रधानमन्त्री नरेन्द्र मोदीले आफ्नी आमा...,1
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,कांग्रेसको १४ औं महाधिवेशमा नेतृत्व चयनका लागि...,1
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,यस वर्ष पहिचान पुरस्कार श्याम शाहको कथासंग्रह ...,1


# droping the column named description

In [12]:
cleaned_data= data.drop(['Description'],axis=1, inplace=False)
cleaned_data.head(10)

,Title,class
0,फिल्म ‘भगवत् गीता’मा धिरज अनुबन्धित,1
1,पब्जी विश्वकपमा नेपाल दोस्रो,1
2,मोदीको मातृभक्ति : आमाको खुट्टा धोएर लिए आशीर्वाद,1
3,‘लोकतन्त्रप्रति प्रतिबद्ध नेतृत्व आउँछ’– निधि,1
4,'पथेर' र 'कदम'लाई पहिचान पुरस्कार,1
5,घाँस खेतीबाट वार्षिक २० लाख रुपियाँ गोकर्णकाे ...,1
6,वैदेशिक रोजगारीका लागि मलेसिया प्रमुख गन्तव्य,1
7,"युक्रेनीलाई बिजुली, पानीबाट वञ्चित गराउँदै रूस",1
8,सुनौलो सहकारीमा पुनः सापकोटा,1
9,अभ्यास खेलमा नेपाल पराजित,1


# suffle gareu data lai

In [14]:
suffle_data = cleaned_data.sample(frac=1)
suffle_data.head(10)

,Title,class
797,सातौं म्याग्दी महोत्सवको तयारी पूरा,1
1278,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0
1492,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0
5236,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0
8800,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1
7238,भीमफेदी–कुलेखानी सुरुङमार्ग : ६ परामर्शदाता छनोट,1
6172,बन्दुक सुरक्षा वार्ताका क्रममा चर्चमा पुरुषले...,0
903,ब्रेकिंग: सेतो जातिवादी संसारमा हुर्किने बारे ...,0
646,रिपब्लिकन मेलरले छिमेकीहरूलाई डेमोक्र्याटलाई भ...,0
4381,फोटोग्राफरले ट्रम्पको नटस्याकको फोटो खिचेपछि इ...,0


# Adding new index

In [16]:
# due to suffleing the index also gets suffeled.
# So reseting new index of the dataframe to its default integer_based indexing.
suffle_data.reset_index(inplace=True)
suffle_data.head()

,index,Title,class
0,797,सातौं म्याग्दी महोत्सवको तयारी पूरा,1
1,1278,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0
2,1492,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0
3,5236,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0
4,8800,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1


In [17]:
# deleting the old index column
suffle_data.drop(['index'],axis=1,inplace = True)
suffle_data.head(5)

,Title,class
0,सातौं म्याग्दी महोत्सवको तयारी पूरा,1
1,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0
2,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0
3,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0
4,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1


# checking null value

In [19]:
suffle_data.isnull().sum()

Title    0
class    0
dtype: int64

# mixing both fake and real into 1

In [21]:
suffle_data.to_csv('mixed.csv', index=False)

In [22]:
data = pd.read_csv('mixed.csv')
data.head()

,Title,class
0,सातौं म्याग्दी महोत्सवको तयारी पूरा,1
1,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0
2,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0
3,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0
4,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1


# Preprocessing

In [24]:
def wordopt(Title):
    
    
    # Title = Title.lower()
    Title = re.sub(r'[\[\]]', '', Title)  # Remove square brackets
    Title = re.sub(r'https?://\S+|www\.\S+', '', Title)  # Remove URLs
    Titel = re.sub(r'<.*?>', '', Title)  # Remove HTML tags
    Title = re.sub(f'[{re.escape(string.punctuation)}]', '', Title)  # Remove punctuation
    Title = re.sub(r'\n', '', Title)  # Remove newlines
    Title = re.sub(r'\w*\d\w*', '', Title)  # Remove words with digits
    Title = re.sub(r'\b[a-zA-Z]+\b', '', Title)  # This removes English words    
    return Title


In [25]:
data['Title2']= data['Title'].apply (wordopt)
data.head(50)

,Title,class,Title2
0,सातौं म्याग्दी महोत्सवको तयारी पूरा,1,सातौं म्याग्दी महोत्सवको तयारी पूरा
1,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0,प्रेसको डरलाग्दो शक्ति अमेरिकीहरूले ओर्लान्डो...
2,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0,ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा धेरै...
3,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...
4,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1,यू क्रिकेटको उपाधि लुम्बिनीलाई
5,भीमफेदी–कुलेखानी सुरुङमार्ग : ६ परामर्शदाता छनोट,1,भीमफेदी–कुलेखानी सुरुङमार्ग परामर्शदाता छनोट
6,बन्दुक सुरक्षा वार्ताका क्रममा चर्चमा पुरुषले...,0,बन्दुक सुरक्षा वार्ताका क्रममा चर्चमा पुरुषले...
7,ब्रेकिंग: सेतो जातिवादी संसारमा हुर्किने बारे ...,0,ब्रेकिंग सेतो जातिवादी संसारमा हुर्किने बारे म...
8,रिपब्लिकन मेलरले छिमेकीहरूलाई डेमोक्र्याटलाई भ...,0,रिपब्लिकन मेलरले छिमेकीहरूलाई डेमोक्र्याटलाई भ...
9,फोटोग्राफरले ट्रम्पको नटस्याकको फोटो खिचेपछि इ...,0,फोटोग्राफरले ट्रम्पको नटस्याकको फोटो खिचेपछि इ...


In [26]:
data.columns

Index(['Title', 'class', 'Title2'], dtype='object')

# Tokenization

In [28]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
data['tokens1'] = data['Title2'].apply(word_tokenize)
data.head()

,Title,class,Title2,tokens1
0,सातौं म्याग्दी महोत्सवको तयारी पूरा,1,सातौं म्याग्दी महोत्सवको तयारी पूरा,"[सातौं, म्याग्दी, महोत्सवको, तयारी, पूरा]"
1,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0,प्रेसको डरलाग्दो शक्ति अमेरिकीहरूले ओर्लान्डो...,"[प्रेसको, डरलाग्दो, शक्ति, अमेरिकीहरूले, ओर्ला..."
2,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0,ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा धेरै...,"[ह्वाइट, हाउसमा, यो, एउटा, कुरालाई, लिएर, ओबाम..."
3,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,"[ट्रम्प, आफ्नै, संयुक्त, प्रमुखहरूद्वारा, अपमा..."
4,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1,यू क्रिकेटको उपाधि लुम्बिनीलाई,"[यू, क्रिकेटको, उपाधि, लुम्बिनीलाई]"


# acessing the list of stopwords

In [30]:
stop = open('stopwords.txt', encoding='utf-8')  # Specify encoding
stopwords = []
for x in stop: # perform through each line of the txt file ma.
    x = x.replace('\n', '')  # Remove newline characters
    stopwords.append(x)      # Add to the stopwords list
stop.close()  # Close the file after reading
data.head(1)

,Title,class,Title2,tokens1
0,सातौं म्याग्दी महोत्सवको तयारी पूरा,1,सातौं म्याग्दी महोत्सवको तयारी पूरा,"[सातौं, म्याग्दी, महोत्सवको, तयारी, पूरा]"


In [31]:
stopwords

['',
 'आफ्न',
 'गर्नेछौ',
 'वाहे',
 'जसम',
 'सङ्गको',
 'चाहे',
 'जस्तै',
 'भनेर',
 'तिमीसँ',
 'चाहन्छौ',
 'जाहि',
 'सम्भ',
 'भन्छन्',
 'त्यस्तो',
 'प्रत्येक',
 'कत',
 'थरि',
 'त्यस्तै',
 'कति',
 'गर्छ',
 'आफैल',
 'सायद',
 'त्यसै',
 'सुरुमै',
 'भीत्र',
 'पटक',
 'अलग',
 'अक्स',
 'गर्दछ',
 'जसको',
 'नत्र',
 'राख्',
 'थिएन',
 'छु',
 'यद',
 'हो',
 'अर्थात',
 'गर्द',
 'छौं',
 'पाँ',
 'होकि',
 'भन्न',
 'दि',
 'नै',
 'पछिल्ल',
 'सध',
 'आत्',
 'पक्क',
 'तेस्कार',
 'पर्नेमा',
 'कम से क',
 'एउटा',
 'सक्नुहुन्छ',
 'भन्द',
 'उनीहरु',
 'म',
 'उसलाई',
 'रे',
 'सम',
 'बिच',
 'त्यत्तिकै',
 'जस्त',
 'पनि',
 'यसैल',
 'वाहेक',
 'ठाउँमा',
 'प्रत्ये',
 'गय',
 'अल',
 'निर्दिष्',
 'कोही',
 'यस',
 'आफैलाई',
 'द्वार',
 'यस्तै',
 'अर्क',
 'मैल',
 'एकद',
 'छिन्',
 'कस्त',
 'देखेको',
 'समयम',
 'तिनै',
 'चाले',
 'पर्याप्त',
 'सम्म',
 'जस',
 'स्पष्',
 'यस्त',
 'अरू',
 'दश',
 'समेत',
 'सोध्',
 'त्य',
 'ऊ',
 'तसरी',
 'मात्र',
 'जबकि',
 'तिनीहरूले',
 'किनभने',
 'उदाहरण',
 'नभएको',
 'केही',
 'साय',
 'तिनीहरूलाई',
 'बाहि

# Removing stopwords

In [33]:
def remove_stopwords(x):
    return ''.join([word for word in str(x).split() if word not in stopwords])
data['title3']= data['tokens1']. apply(lambda x :remove_stopwords(x))# for each row in the tokens column 
#lambda function is called with x being the value of that row.
#The result of the lambda function (which is the value returned by remove_stopwords(x)) is assigned to the new column title3.
data.head(5)

,Title,class,Title2,tokens1,title3
0,सातौं म्याग्दी महोत्सवको तयारी पूरा,1,सातौं म्याग्दी महोत्सवको तयारी पूरा,"[सातौं, म्याग्दी, महोत्सवको, तयारी, पूरा]","['सातौं','म्याग्दी','महोत्सवको','तयारी','पूरा']"
1,प्रेसको डरलाग्दो शक्ति: 41% अमेरिकीहरूले ओर्ला...,0,प्रेसको डरलाग्दो शक्ति अमेरिकीहरूले ओर्लान्डो...,"[प्रेसको, डरलाग्दो, शक्ति, अमेरिकीहरूले, ओर्ला...","['प्रेसको','डरलाग्दो','शक्ति','अमेरिकीहरूले','..."
2,"ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा ""धेर...",0,ह्वाइट हाउसमा यो एउटा कुरालाई लिएर ओबामा धेरै...,"[ह्वाइट, हाउसमा, यो, एउटा, कुरालाई, लिएर, ओबाम...","['ह्वाइट','हाउसमा','यो','एउटा','कुरालाई','लिएर..."
3,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,0,ट्रम्प आफ्नै संयुक्त प्रमुखहरूद्वारा अपमानित ह...,"[ट्रम्प, आफ्नै, संयुक्त, प्रमुखहरूद्वारा, अपमा...","['ट्रम्प','आफ्नै','संयुक्त','प्रमुखहरूद्वारा',..."
4,यू-१९ क्रिकेटको उपाधि लुम्बिनीलाई,1,यू क्रिकेटको उपाधि लुम्बिनीलाई,"[यू, क्रिकेटको, उपाधि, लुम्बिनीलाई]","['यू','क्रिकेटको','उपाधि','लुम्बिनीलाई']"


# Stemming the language

In [35]:
# import snowballstemmer
# test_words = ["नेपालीको", "आफ्नै", "स्तेम्मेर", "यसलाई", "अनुमोदनको"]
#  stemmed_words = nepali_stemmer.stemWords(test_words)
#  print("Original Words:", test_words)
#  print("Stemmed Words:", stemmed_words)


In [36]:
# import re
# import snowballstemmer

# # Initialize the NepaliStemmer instance
# nepali_stemmer = snowballstemmer.NepaliStemmer()

# def preprocess_text(text):
#     if isinstance(text, str):
#         # Remove punctuation and extra spaces
#         text = re.sub(r'[^\w\s]', '', text)
#         return text.strip()
#     return text

# def stem_text(text):
#     text = preprocess_text(text)
#     if isinstance(text, str):
#         return " ".join(nepali_stemmer.stemWords(text.split()))
#     return text

# # Apply the preprocessing and stemming to the 'title3' column
# data['data4'] = data['title3'].apply(stem_text)

# # Display results
# print(data[['title3', 'data4']].head(20))
# data.head(20)

In [37]:
data.columns

Index(['Title', 'class', 'Title2', 'tokens1', 'title3'], dtype='object')

In [38]:
x = data['title3']
y = data['class']

In [39]:
x

0          ['सातौं','म्याग्दी','महोत्सवको','तयारी','पूरा']
1        ['प्रेसको','डरलाग्दो','शक्ति','अमेरिकीहरूले','...
2        ['ह्वाइट','हाउसमा','यो','एउटा','कुरालाई','लिएर...
3        ['ट्रम्प','आफ्नै','संयुक्त','प्रमुखहरूद्वारा',...
4                 ['यू','क्रिकेटको','उपाधि','लुम्बिनीलाई']
                               ...                        
19509    ['बाढीपहिरो','र','डुबानमा','परी','हालसम्म','जन...
19510         ['मरिसनले','दिए','नयाँ','वर्षको','शुभकामना']
19511    ['यो','तस्विरलाई','इयरबुकमा','नआउनुको','प्रिन्...
19512                   ['घरघरै','बिरामी','छैन','परीक्षण']
19513                  ['लिटरेचर','फेस्टिभल','पुस','देखि']
Name: title3, Length: 19514, dtype: object

In [40]:
y

0        1
1        0
2        0
3        0
4        1
        ..
19509    1
19510    1
19511    0
19512    1
19513    1
Name: class, Length: 19514, dtype: int64

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.10)

In [42]:
x_train.shape

(17562,)

In [43]:
x_test.shape

(1952,)

In [44]:
x_train.head(50)

13892    ['स्वास्थ्य','संकटकाल','जरुरी','भइसक्यो','डासन...
14284    ['अलोकतान्त्रिक','सरकारलाई','ठेगान','लगाउने','...
10073    ['ट्रम्प','यति','पागल','छन्','कि','उनले','आफ्न...
136      ['प्रतिशत','कोरोना','सङ्क्रमितमा','लक्षण','देख...
2310     ['ओभल','अफिसबाट','हटाइएको','एमएलके','जेआरको','...
4514     ['भूतपूर्व','ग्रान्ड','विजार्ड','कालो','पुरुषक...
9425     ['नियुक्त','भएको','केही','घण्टामै','किन','दिइन...
3074     ['साढे','वर्ष','जेल','बसेपछि','मुद्दा','झुटो',...
204          ['भेरी–बबईमा','प्रतिदिन','तीन','चरणमा','काम']
14268    ['पर्दाफास','मेलानिया','ट्रम्प','व्यक्तिगत','ल...
11243    ['टकर','कार्लसन','शरणार्थी','समर्थक','नेतामा',...
3137            ['सर्लाहीसँग','हिमालयन','शेर्पा','स्तब्ध']
5119     ['टीकापुरको','पुरानो','सडक','स्तरोन्नती','गरिद...
12790    ['मृत्यु','र','कर','राजकुमारको','आधा','भन्दा',...
9643     ['डेभिड','फ्रान्सेलीलाई','भेट्नुहोस्','डोनाल्ड...
7737     ['जलाउनुहोस्','ले','मा','झडप','लिन्छहेर्नुहोस्...
5637     ['राईको','‘','तेप्लायु','’','लाई','विमल','कृति.

In [45]:
y_test.head(50)

16461    1
9247     0
6695     0
6422     0
3676     1
9790     0
15191    1
10357    0
7381     0
8648     0
16515    1
14757    0
11587    1
2796     0
17391    0
860      1
11175    1
14692    0
12397    0
12859    1
3573     1
18474    0
4668     0
11500    1
17319    1
18776    1
3923     1
13573    0
10203    1
14626    0
3972     1
5883     0
12268    1
19223    0
10631    1
9668     0
15518    0
14445    1
1530     0
11547    1
3243     1
12068    1
18277    1
14883    0
10968    0
10569    0
3117     1
14416    1
5679     1
2952     0
Name: class, dtype: int64

In [46]:
# here i built different file which contain only testing data
x_test.to_csv('testing_data.csv', index=False)

In [47]:
# similarly building file for training data
x_train.to_csv('training_data.csv', index=False)

# Vectorizing the text data into numerical


In [49]:
# #  here due to the nepali datasets the system doest tfidf the data.
# # so for that i have to convert the data into numerical featues using countvectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(ngram_range=(1,2)).fit(x_train)
# x_train_vectorized = vectorizer.transform(x_train)
# print(x_train_vectorized)
# x_train_vectorized.shape

In [50]:
# vectorizer = CountVectorizer(ngram_range=(1,2))
# x_train_vectorized = vectorizer.fit_transform(x_train)

# Transforming the text data

In [52]:
#converting the text into vector The TfidfVectorizer is used to convert a collection of text documents into numerical vectors,
#that can be fed into machine learning models. It does this by calculating the TF-IDF for each word in the documents.
vectorization = TfidfVectorizer() 
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [53]:
xv_train

<17562x4033 sparse matrix of type '<class 'numpy.float64'>'
	with 136848 stored elements in Compressed Sparse Row format>

# Model Training

In [55]:
model = LogisticRegression()
model.fit(xv_train,y_train)

LogisticRegression()

In [56]:
pred_lr=model.predict(xv_test)

In [57]:
model.score(xv_test, y_test)

0.9267418032786885

In [58]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       998
           1       0.92      0.94      0.93       954

    accuracy                           0.93      1952
   macro avg       0.93      0.93      0.93      1952
weighted avg       0.93      0.93      0.93      1952



In [59]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"

def manual_testing(news):
    # Create a DataFrame from the input news
    testing_news = {"title3": [news]}
    new_def_test = pd.DataFrame(testing_news)

    # Apply preprocessing using wordopt (assuming it's a defined function)
    new_def_test["title3"] = new_def_test["title3"].apply(wordopt) 

    # Transform the text using vectorization
    new_x_test = new_def_test["title3"]
    new_xv_test = vectorization.transform(new_x_test)

    # Perform prediction using Logistic Regression
    pred_model = model.predict(new_xv_test)

    # Output the prediction result
    return print("\n\nLogistic Regression Prediction: {}".format(output_label(pred_model[0])))


In [61]:
news_input = "'सिनेमाघरमा','‘','मिलन','’','र','‘','भेडिया',.."
manual_testing(news_input)



Logistic Regression Prediction: Real News


# saving the model using pickle

In [64]:
import pickle


In [66]:
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [68]:
# load saved model
with open('model_pkl' , 'rb') as file:
    model= pickle.load(file)

In [78]:
input_text = ["'सिनेमाघरमा','‘','मिलन','’','र','‘','भेडिया',.."]
input_vectorized = vectorization.transform(input_text)

In [80]:
# model.predict([["'राष्ट्रपति','पदका','उम्मेद्वार','मार्को','रु.."]])
prediction = model.predict(input_vectorized)
print(prediction)

[1]
